In [ ]:
from run_rag import RAGApp

a = RAGApp("datasets\manual.pdf")
a.set_rag_model()
response = a.ask_question("박사 과정의 월급은?")
print(response)

In [1]:
import sys
import requests
from PyQt6.QtWidgets import (
    QApplication, QWidget, QVBoxLayout, QPushButton, QLabel, QFileDialog
)
from PyQt6.QtCore import Qt

class DragDropLabel(QLabel):
    """드래그 앤 드롭을 지원하는 커스텀 라벨 위젯"""
    def __init__(self, parent=None):
        super().__init__(parent)
        self.setAcceptDrops(True)
        self.setAlignment(Qt.AlignmentFlag.AlignCenter)
        self.setText("여기에 PDF 파일을 드래그 앤 드롭 하거나\n아래 버튼을 클릭하세요.")
        self.setStyleSheet("""
            QLabel {
                border: 2px dashed #aaa;
                padding: 20px;
                font-size: 14px;
            }
        """)

    def dragEnterEvent(self, event):
        # 드래그된 데이터에 URL(파일 경로)이 포함되어 있는지 확인
        if event.mimeData().hasUrls():
            event.acceptProposedAction()
        else:
            event.ignore()

    def dropEvent(self, event):
        # 드롭된 파일의 경로를 가져옴
        for url in event.mimeData().urls():
            file_path = url.toLocalFile()
            # PDF 파일만 처리
            if file_path.lower().endswith('.pdf'):
                print(f"드롭된 파일: {file_path}")
                self.parent().upload_file(file_path)
                return
        print("PDF 파일이 아닙니다.")


class PDFUploader(QWidget):
    def __init__(self):
        super().__init__()
        self.init_ui()
        # Flask 서버 주소
        self.server_url = "http://127.0.0.1:5000/upload"

    def init_ui(self):
        layout = QVBoxLayout()

        # 드래그 앤 드롭 라벨
        self.drop_label = DragDropLabel(self)
        layout.addWidget(self.drop_label)

        # 파일 선택 버튼
        self.btn_select = QPushButton("컴퓨터에서 PDF 파일 선택")
        self.btn_select.clicked.connect(self.open_file_dialog)
        layout.addWidget(self.btn_select)
        
        # 상태 메시지 라벨
        self.status_label = QLabel("상태: 대기 중")
        self.status_label.setAlignment(Qt.AlignmentFlag.AlignCenter)
        layout.addWidget(self.status_label)

        self.setLayout(layout)
        self.setWindowTitle("PyQt PDF Uploader")
        self.setGeometry(300, 300, 400, 250)

    def open_file_dialog(self):
        # 파일 대화상자를 열어 PDF 파일 선택
        file_path, _ = QFileDialog.getOpenFileName(self, "PDF 파일 선택", "", "PDF Files (*.pdf)")
        if file_path:
            self.upload_file(file_path)

    def upload_file(self, file_path):
        self.status_label.setText(f"상태: '{file_path.split('/')[-1]}' 업로드 중...")
        
        try:
            with open(file_path, 'rb') as f:
                files = {'pdf_file': (file_path.split('/')[-1], f, 'application/pdf')}
                response = requests.post(self.server_url, files=files, timeout=10)
            
            # 응답 상태 확인
            if response.status_code == 200:
                response_data = response.json()
                self.status_label.setText(f"상태: {response_data.get('message', '성공!')}")
            else:
                response_data = response.json()
                self.status_label.setText(f"상태: 서버 오류 - {response_data.get('error', '알 수 없는 오류')}")

        except requests.exceptions.RequestException as e:
            self.status_label.setText(f"상태: 전송 오류 - {e}")
        except Exception as e:
            self.status_label.setText(f"상태: 오류 발생 - {e}")

if __name__ == '__main__':
    # 필요한 라이브러리 설치 확인
    try:
        import requests
    except ImportError:
        print("requests 라이브러리가 필요합니다. 'pip install requests' 명령어로 설치해주세요.")
        sys.exit(1)
        
    app = QApplication(sys.argv)
    ex = PDFUploader()
    ex.show()
    sys.exit(app.exec())

SystemExit: 0

c:\Users\minja\anaconda3\envs\rag\lib\site-packages\IPython\core\interactiveshell.py:3513: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
